In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
df=spark.read.csv('cancerdata_test.csv',header=True,inferSchema=True)




FileNotFoundError: [Errno 2] No such file or directory: '/opt/spark/./bin/spark-submit'

In [5]:
df=df.drop('_c32').drop('id')
df.show()

+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+
|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|
+---------+-----------+------------+--------------+---------+-----

In [9]:
from pyspark.ml.feature import VectorAssembler
va=VectorAssembler(inputCols=([c for c in df.columns if c!='diagnosis']),outputCol='features')

In [10]:
vaDf=va.transform(df)


In [11]:
vaDf.select('features','diagnosis').show()

+--------------------+---------+
|            features|diagnosis|
+--------------------+---------+
|[17.99,10.38,122....|        M|
|[20.57,17.77,132....|        M|
|[19.69,21.25,130....|        M|
|[11.42,20.38,77.5...|        M|
|[20.29,14.34,135....|        M|
|[12.45,15.7,82.57...|        M|
|[18.25,19.98,119....|        M|
|[13.71,20.83,90.2...|        M|
|[13.0,21.82,87.5,...|        M|
|[12.46,24.04,83.9...|        M|
|[16.02,23.24,102....|        M|
|[15.78,17.89,103....|        M|
|[19.17,24.8,132.4...|        M|
|[15.85,23.95,103....|        M|
|[13.73,22.61,93.6...|        M|
|[14.54,27.54,96.7...|        M|
|[14.68,20.13,94.7...|        M|
|[16.13,20.68,108....|        M|
|[19.81,22.15,130....|        M|
|[13.54,14.36,87.4...|        B|
+--------------------+---------+
only showing top 20 rows



In [12]:
#scaling
from pyspark.ml.feature import StandardScaler

scaler=StandardScaler(inputCol='features',outputCol='scaledFeatures')
scaledDf=scaler.fit(vaDf).transform(vaDf)




In [29]:
#encoding
from pyspark.ml.feature import StringIndexer
sInd=StringIndexer(inputCol='diagnosis',outputCol='label')
df=sInd.fit(scaledDf).transform(scaledDf).select('scaledFeatures','label')
df.show(truncate=False)



+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|scaledFeatures                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [30]:
from pyspark.ml.classification import DecisionTreeClassifier
dt=DecisionTreeClassifier(
featuresCol='scaledFeatures',
labelCol='label',
predictionCol='prediction',
probabilityCol='probability',
rawPredictionCol='rawPrediction',
maxDepth=30,
maxBins=15,
minInstancesPerNode=5,
minInfoGain=0.0,
maxMemoryInMB=256,
cacheNodeIds=False,
checkpointInterval=-1,
impurity='gini',
seed=None,

)


In [31]:
train,test=df.randomSplit([0.8,0.2])

In [32]:
dtModel=dt.fit(train)

In [34]:
predicteDf=dtModel.transform(test)



In [36]:
predicteDf.count()

109

In [41]:
#-----------------------------------------------------Evaulator of model
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(
    rawPredictionCol='rawPrediction',
    labelCol='label',
    metricName='areaUnderROC',
 
)
result=evaluator.evaluate(predicteDf)
result



0.8882373845060413

In [46]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
mevaulator=MulticlassClassificationEvaluator(
    predictionCol='prediction',
    labelCol='label',
    metricName='f1',
    metricLabel=0.0,
    probabilityCol='probability',

    
)

In [47]:
result=mevaulator.evaluate(predicteDf)
result

0.9361517154441898